# EfficientNet-B2

In [1]:
# Load the TensorBoard notebook extension.
%load_ext tensorboard

import os
import random
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import cv2
import math
import scipy
from datetime import datetime
from packaging import version
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import TensorBoard

from tensorflow_model.efficientnet import EfficientNet

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.3.0 or above."

%pylab inline

TensorFlow version:  2.3.0
Populating the interactive namespace from numpy and matplotlib


C:\Users\brand\miniconda3\envs\carnd\lib\site-packages\IPython\core\magics\pylab.py:160: UserWarning: pylab import has clobbered these variables: ['datetime', 'random']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
# Change to other families by referencing: 
# ~\miniconda3\envs\carnd\Lib\site-packages\tensorflow\python\keras\applications\efficientnet.py

def EfficientNetB2(include_top=True,
                   weights='imagenet',
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=1000,
                   classifier_activation='softmax',
                   **kwargs):
    return EfficientNet(
        1.1,
        1.2,
        260,
        0.3,
        model_name='efficientnetb2',
        include_top=include_top,
        weights=weights,
        input_tensor=input_tensor,
        input_shape=input_shape,
        pooling=pooling,
        classes=classes,
        classifier_activation=classifier_activation,
        **kwargs)

In [3]:
img_input_data = keras.layers.Input(shape=(6, 128, 256), name="input_imgs")
permute1 = keras.layers.Permute((2,3,1))(img_input_data)
efficientnet = EfficientNetB2(include_top=False, weights = None, input_shape=(128,256,6))(permute1)
conv2d1 = tf.keras.layers.Conv2D(32, 3, padding='same')(efficientnet)
normalization = keras.layers.BatchNormalization()(conv2d1)
elu1 = tf.keras.layers.ELU(alpha=1.0)(normalization)
flatten1 = tf.keras.layers.Flatten(name="vision_features")(elu1)


dense1 = tf.keras.layers.Dense(512)(flatten1)
dense1_relu = tf.keras.layers.ReLU(name="dense1_relu")(dense1)
#droupout1 = tf.keras.layers.Dropout(0.2)(dense1_relu)
dense2 = tf.keras.layers.Dense(256)(dense1_relu)
dense2_relu = tf.keras.layers.ReLU(name="dense2_relu")(dense2)
#droupout2 = tf.keras.layers.Dropout(0.2)(dense2_relu)
path = tf.keras.layers.Dense(100)(dense2_relu)

output = path
model = keras.models.Model(inputs=[img_input_data], outputs=[output])

# Renaming all layers which deals with image
"""
for layer in model.layers:
    layer._name = "vision_" + layer.name
"""
    
#concat = keras.layers.concatenate([input_data, hidden2])

model.compile(optimizer='adam', loss="mse", metrics=['accuracy'])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_imgs (InputLayer)      [(None, 6, 128, 256)]     0         
_________________________________________________________________
permute (Permute)            (None, 128, 256, 6)       0         
_________________________________________________________________
efficientnetb2 (Functional)  (None, 4, 8, 1408)        7769439   
_________________________________________________________________
conv2d (Conv2D)              (None, 4, 8, 32)          405536    
_________________________________________________________________
batch_normalization (BatchNo (None, 4, 8, 32)          128       
_________________________________________________________________
elu (ELU)                    (None, 4, 8, 32)          0         
_________________________________________________________________
vision_features (Flatten)    (None, 1024)             

### Import data

In [4]:
DATA_PATH = "training_data/baseline_batch/"
x_training = np.load(DATA_PATH + "input.npy")
y_training = np.load(DATA_PATH + "output.npy")

In [5]:
# Model checkpoint
checkpoint = tf.keras.callbacks.ModelCheckpoint('efficientnet_v1.h5', verbose=1, save_best_only=True, mode='min')

callbacks = [tf.keras.callbacks.EarlyStopping(patience=200, monitor='val_loss'), tf.keras.callbacks.TensorBoard(log_dir='logs')]

results = model.fit(x_training, y_training, validation_split=0.10, batch_size=8, epochs=500, callbacks=callbacks)
model.save("efficientnet_v1.h5")

Epoch 1/500
 1/69 [..............................] - ETA: 0s - loss: 4.9223 - accuracy: 0.0000e+00WARNING:tensorflow:From C:\Users\brand\miniconda3\envs\carnd\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
69/69 [==============================] - 26s 382ms/step - loss: 2.4962 - accuracy: 0.0891 - val_loss: 0.8077 - val_accuracy: 0.0000e+00
Epoch 2/500
69/69 [==============================] - 47s 675ms/step - loss: 2.1896 - accuracy: 0.1636 - val_loss: 0.4542 - val_accuracy: 0.0000e+00
Epoch 3/500
69/69 [==============================] - 122s 2s/step - loss: 1.8107 - accuracy: 0.1364 - val_loss: 108.2143 - val_accuracy: 0.0000e+00
Epoch 4/500
69/69 [==============================] - 106s 2s/step - loss: 1.8613 - accuracy: 0.2400 - val_loss: 0.3865 - val_accuracy: 0.0000e+00
Epoch 5/500
69/69 [===========

69/69 [==============================] - 47s 677ms/step - loss: 0.0314 - accuracy: 0.6073 - val_loss: 0.2092 - val_accuracy: 1.0000
Epoch 54/500
69/69 [==============================] - 45s 648ms/step - loss: 0.0414 - accuracy: 0.5800 - val_loss: 0.1408 - val_accuracy: 0.9194
Epoch 55/500
69/69 [==============================] - 45s 657ms/step - loss: 0.0386 - accuracy: 0.5800 - val_loss: 0.1665 - val_accuracy: 0.5000
Epoch 56/500
69/69 [==============================] - 68s 987ms/step - loss: 0.0367 - accuracy: 0.5782 - val_loss: 0.2261 - val_accuracy: 0.8065
Epoch 57/500
69/69 [==============================] - 94s 1s/step - loss: 0.0587 - accuracy: 0.5836 - val_loss: 0.2400 - val_accuracy: 1.0000
Epoch 58/500
69/69 [==============================] - 67s 975ms/step - loss: 0.0607 - accuracy: 0.5655 - val_loss: 0.1801 - val_accuracy: 0.8548
Epoch 59/500
69/69 [==============================] - 57s 829ms/step - loss: 0.1434 - accuracy: 0.4836 - val_loss: 0.1857 - val_accuracy: 0.9677
E

Epoch 110/500
69/69 [==============================] - 67s 972ms/step - loss: 0.0097 - accuracy: 0.6455 - val_loss: 0.2003 - val_accuracy: 0.9516
Epoch 111/500
69/69 [==============================] - 60s 873ms/step - loss: 0.0189 - accuracy: 0.6036 - val_loss: 0.1434 - val_accuracy: 1.0000
Epoch 112/500
69/69 [==============================] - 53s 773ms/step - loss: 0.0145 - accuracy: 0.6291 - val_loss: 0.1838 - val_accuracy: 0.9677
Epoch 113/500
69/69 [==============================] - 55s 793ms/step - loss: 0.0129 - accuracy: 0.6345 - val_loss: 0.2973 - val_accuracy: 0.9355
Epoch 114/500
69/69 [==============================] - 57s 825ms/step - loss: 0.0113 - accuracy: 0.6400 - val_loss: 0.2946 - val_accuracy: 1.0000
Epoch 115/500
69/69 [==============================] - 64s 924ms/step - loss: 0.0123 - accuracy: 0.6345 - val_loss: 0.2774 - val_accuracy: 0.9516
Epoch 116/500
69/69 [==============================] - 91s 1s/step - loss: 0.0174 - accuracy: 0.6545 - val_loss: 0.3339 - va

69/69 [==============================] - 64s 926ms/step - loss: 0.0060 - accuracy: 0.6891 - val_loss: 0.2518 - val_accuracy: 1.0000
Epoch 167/500
69/69 [==============================] - 46s 663ms/step - loss: 0.0074 - accuracy: 0.6818 - val_loss: 0.3487 - val_accuracy: 1.0000
Epoch 168/500
69/69 [==============================] - 46s 663ms/step - loss: 0.0146 - accuracy: 0.6909 - val_loss: 0.2498 - val_accuracy: 1.0000
Epoch 169/500
69/69 [==============================] - 49s 705ms/step - loss: 0.0126 - accuracy: 0.6727 - val_loss: 0.3935 - val_accuracy: 1.0000
Epoch 170/500
69/69 [==============================] - 51s 744ms/step - loss: 0.0092 - accuracy: 0.6709 - val_loss: 0.2469 - val_accuracy: 1.0000
Epoch 171/500
69/69 [==============================] - 71s 1s/step - loss: 0.0088 - accuracy: 0.6691 - val_loss: 0.2457 - val_accuracy: 1.0000
Epoch 172/500
69/69 [==============================] - 68s 983ms/step - loss: 0.0075 - accuracy: 0.7018 - val_loss: 0.2449 - val_accuracy: 0.

# Enable import from different directory
import sys
sys.path.insert(0, 'D:/comma2k19_dataset/notebooks/lib/')
from common.transformations.camera import transform_img, eon_intrinsics
from common.transformations.model import medmodel_intrinsics


VIDEO_PATH = 'D:/comma2k19_dataset/10/video.hevc'

TOTAL_FRAMES = 1200
bgr_imgs = []
yuv_imgs = []


vid = cv2.VideoCapture(VIDEO_PATH)
for frame_number in tqdm(range(TOTAL_FRAMES)):
    ret, frame = vid.read()
    if (frame_number % 5 == 0):
        bgr_imgs.append(frame)
        frame = cv2.resize(frame,(1164,874))
        img_yuv = cv2.cvtColor(frame, cv2.COLOR_BGR2YUV_I420)
        yuv_imgs.append(img_yuv)



def frames_to_tensor(frames):                                                                                               
    H = (frames.shape[1]*2)//3                                                                                                
    W = frames.shape[2]                                                                                                       
    in_img1 = np.zeros((frames.shape[0], 6, H//2, W//2), dtype=np.uint8)                                                      
                                                                                                                            
    in_img1[:, 0] = frames[:, 0:H:2, 0::2]                                                                                    
    in_img1[:, 1] = frames[:, 1:H:2, 0::2]                                                                                    
    in_img1[:, 2] = frames[:, 0:H:2, 1::2]                                                                                    
    in_img1[:, 3] = frames[:, 1:H:2, 1::2]                                                                                    
    in_img1[:, 4] = frames[:, H:H+H//4].reshape((-1, H//2,W//2))                                                              
    in_img1[:, 5] = frames[:, H+H//4:H+H//2].reshape((-1, H//2,W//2))
    return in_img1

imgs_med_model = np.zeros((len(yuv_imgs), 384, 512), dtype=np.uint8)
for i, img in tqdm(enumerate(yuv_imgs)):
    imgs_med_model[i] = transform_img(img, from_intr=eon_intrinsics, to_intr=medmodel_intrinsics, yuv=True,
                                    output_size=(512,256))
frame_tensors = frames_to_tensor(np.array(imgs_med_model)).astype(np.float32)/128.0 - 1.0

figsize(12,12);
imshow(bgr_imgs[0]);
title('First Image', fontsize=25);

outs = model.predict(frame_tensors, verbose=1)
data_length = len(outs)

verification_folder = "validation"
# Get the length of the data

for i in tqdm(range(data_length)):
    verification_img = bgr_imgs[i]
    forward_data = np.linspace(0,99,100)
    down_data = np.zeros(100)
    draw_path(np.transpose([forward_data,outs[i],down_data]), img=verification_img, isTest = True)
        
    cv2.imwrite(os.path.join(verification_folder , str(i) + '.jpg'), verification_img) 